# 积分

在数学定义中，积分(integration)和微分(differentiation)都涉及到了极限运算。但，计算机的精度有限，并不能处理无限小与无限大的运算，最多只能模拟求极限的过程。在本章教学中，我们先简要讨论数值微分与积分问题，之后再讨论计量经济学中随机方法的具体应用。其中我们会特别介绍模拟矩方法(simulated method of moments)，间接推断(indirect inference)和马尔可夫链-蒙特卡罗法(Markov Chain Monte Carlo, MCMC)。这些方法超出了课程 Econ5121A 和 Econ5150 的授课范围， 感兴趣的读者可以参考{cite}`cameron2005microeconometrics`的第12与13章节以获得更详细的内容。



## 数理方法

数值微分和积分是非常重要的基础知识。不过，电脑如何进行这些运算与经济学或者计量经济学并无关系；实际上，它更应该是一个数据分析课程的教学内容。因此，在这里我们快速地过一遍这些数学方法。{cite}`judd1998numerical`是一本可靠的参考文献，相关知识可以详见此书第7章。 

在本科的微积分教学中，我们已经学过了许多常见函数的微分。然而，在某些情况下这些解析形式的微分不能或很难程序编译。例如，如果想要用牛顿法求出目标函数 $f:R^K \mapsto R$ 的极值，理论上我们需要编辑它的 $K$ 维梯度和 $K\times K$ 海塞矩阵。而写出海塞矩阵就是一个费时费力还容易出错的工作。更甚的是，每当我们改变目标函数(这在测验和试错阶段时常发生)，我们就必须重新计算相应的梯度和海塞矩阵。所以，相较于写出解析表达式，数值微分无疑更加方便省事。尤其在早期的测试阶段，这种效率的差距就更加显著了。

多元函数 $f:R^K \mapsto R$ at a point $x_0 \in R^K$ 的偏导为

$$
\frac{\partial f(x)}{\partial x_k}\bigg|_{x=x_0}=\lim_{\epsilon \to 0}
\frac{f(x_0+\epsilon \cdot e_k) - f(x_0 - \epsilon \cdot e_k)}{2\epsilon},
$$

其中 $e_k = (0,\ldots,0,1,0,\ldots,0)$ 是第 $k$ 阶坐标的符号(identifier)。
计算机的数值运算方法是计算 $\epsilon$ 非常小的
$f(x_0\pm\epsilon \cdot e_k))$ 。但是，怎么知道多小才算 **小** ？ 常规操作是尝试一系列递减的 $\epsilon$ 直到导数的数值解趋于稳定。有许多的随机算法可用于处理这种问题。

在R中，拓展包 `numDeriv` 就能够计算数值微分，其中

* `grad` 用于标量值函数
* `jacobian` 用于实数向量值函数
* `hessian` 用于标量值函数
* `genD` 用于实数向量值函数

从总体上看，积分运算比微分更难。在R中，`integrate` 可以用于计算一维求积，而`cubature`中的
`adaptIntegrate`可以处理多维求积分问题。读者可以自行查阅数值积分的算法操作手册。

当然，数值法并非万能灵药。并非所有方程都可微或者可积。在进行数值运算前，必须要先理解函数的行为(behavior)。尽管R语言中有一些能够进行符号运算的拓展包，但是R本身符号运算的功能很薄弱。`Mathematica` 或者 `Wolfram Alpha`，是进行符号运算的实用工具。





## 随机方法

数值积分的另一种方式是 **随机方法** (stochastic methods)。随机积分背后的原理是大数定律。
令 $\int h(x) d F(x)$ 为 $F(x)$ 的积分。( $F(x)$ 是概率分布函数）
用函数 $\int h(x) d F(x) \approx S^{-1} \sum_{s=1}^S h(x_s)$ ，我们就可以估计积分值，其中 $x_s$ 由 $F(x)$ 随机生成。当 $S$ 足够大时，大数定律就可以得出:

$$
S^{-1} \sum_{s=1}^S h(x_s) \stackrel{\mathrm{p}}{\to} E[h(x)] = \int h(x) d F(x).
$$

如果积分并非由整体 $F(x)$ 而由子集 $A$ 得到，那么

$$
\int_A h(x) d F(x) \approx S^{-1} \sum_{s=1}^S h(x_s) \cdot 1\{x_s \in A\},
$$

其中 $1\{\cdot\}$ 是指示函数。

**指示函数** ：指示函数用来表示有哪些元素满足特定条件或者属于某一子集A，有时候也被称为特征函数。

理论上我们希望 $S$ 越大越好，但现实操作则往往受到计算机的储存和运算时间的限制。运算中 $S$ 的大小没有统一的规定，我们可以通过初步实验来估计运算所需要的 $S$ 大小。

由于操作便捷、使用简单，随机积分法在计量经济学和统计学中广受欢迎。

**例子**

在经济学模型中，可能会遇到部分元素无法观测的问题，这将会影响到经济主体的决策。
{cite}`roy1951some`提出了一个隐变量模型(Roy模型)，它是劳动经济学中自选择问题的基础。

Roy模型的原文中，经济主体必须是农民或者渔民。农民的效用函数被定义为 $U_1^{\star} = x' \beta_1 + e_1$ ，渔民则是
 $U_2^{\star} = x' \beta_2 + e_2$ ，
其中 $U_1^{\star}$ 和 $U_2^{\star}$ 是隐变量 (不可观测)。计量学者可观测到的是
 $y=\mathbf{1}\{U_1^{\star}> U_2^{\star}\}$ 。如果
$(e_1,e_2)$ 独立于 $x$ ，且

$$
\begin{bmatrix}
e_1\\e_2
\end{bmatrix}
\sim N \left(
\begin{bmatrix}
0 \\ 0
\end{bmatrix},
  \begin{bmatrix}
  \sigma_1^2 & \sigma_{12} \\ \sigma_{12} & 1
  \end{bmatrix}\right)
$$

其中 $\sigma_2$ 被正态标准化为1， 我们可以写出对数似然

$$
L(\theta) = \sum_{i = 1}^n  \left\{ y_i \log P( U_{i1}^{\star} > U_{i2}^{\star} )
+ (1-y_i)\log P( U_{i1}^* \leq  U_{i2}^{\star} ) \right\}.
$$

令 $\theta = (\beta_1, \beta_2, \sigma_1, \sigma_{12})$ 。
给定测试值 $\theta$ ，我们就可以计算

$$
p(\theta|x_i) = P\left( U^{\star}_{i1}(\theta) > U^{\star}_{i2}(\theta) \right)
= P\left( x_i'(\beta_1 - \beta_2) > e_{i2} -e_{i1} \right).
$$

在联合正态分布假设下， $e_{i2} - e_{i1} \sim N(0, \sigma_1^2 - 2\sigma_{12}+1)$ ，因此

$$
p(\theta|x_i) = \Phi \left(  \frac{x_i'(\beta_1 - \beta_2)}{\sqrt{\sigma_1^2 - 2\sigma_{12}+1}} \right)
$$

其中 $\Phi(\cdot)$ 是标准正态分布的累积分布函数(CDF)。

不过，注意到它的解析形式依赖于联合正态分布的假设，并不能随便推广到其他的分布。但只要电脑能够生成 $(e_{i1}, e_{i2})$ 的联合分布，无论是否正态，我们都可以使用随机方法。我们估计：

$$
\hat{p}(\theta|x_i) = \frac{1}{S} \sum_{i=1}^S \mathbf{1}\left( U^{s*}_{i1}(\theta) > U^{s*}_{i2}(\theta) \right),
$$

其中 $s=1,\ldots,S$ 是模拟的下标， $S$ 是模拟复制的总数量。

接下来，我们将理论模型生成的矩与它们的真实值对应。选取哪些矩由使用者依据情况自行决定。Roy模型的一些常用选择是：

$$
\begin{align*}
g_1(\theta) & =  n^{-1} \sum_{i=1}^n x_i (y_i - \hat{p}(\theta | x_i))  \approx  0\\
g_2(\theta) & =  n^{-1} \sum_{i=1}^n (y_i - \bar{y})^2 - \bar{\hat{p}}(\theta| x_i) (1- \bar{\hat{p}}(\theta| x_i))  \approx  0\\
g_3(\theta) & =  n^{-1} \sum_{i=1}^n (x_i - \bar{x} ) (y_i - \hat{p}(\theta | x_i))^2  \approx  0
\end{align*}
$$

其中 $\bar{y} = n^{-1} \sum_{i=1}^n y_i$ 且 $\bar{\hat{p}}(\theta) = n^{-1} \sum_{i=1}^n p(\theta|x_i)$ 。
由于 $(e_{i1}, e_{i2})$ 与 $x_i$ 独立，我们就能得到第一组矩，进而得到 $E[x_i y_i] = x_i E[y_i | x_i] = x_i p(\theta|x_i)$ 。
第二组与 $y_i$ 的方差相对应。由于矩条件 $(g_j(\theta))_{j=1}^3$ 等于未知参数的数量，这些矩条件恰好识别参数 $\theta$ 。我们需要找出更多的矩来进行过度识别。此外，我们需要选择一个权重矩阵 $W$ 来形成GMM的二次型目标函数。

上面的这个例子是模拟最大似然(simulated maximum likelihood)的其中一个应用。如果无法写出解析形式，则可以模拟一个矩条件。
{cite}`pakes1989simulation`给出了模拟矩估计(simulated method of moments, SMM）的理论支撑。我们的另一位老师Prof. Guo [{cite}`guo2018`] 最近将SMM应用于一个结构化劳动力模型。



### 间接推断

间接推断(indirect inference) [{cite}`smith1993estimating`, {cite}`gourieroux1993indirect`]
是另一个基于模拟的估计方法。间接推断在结构化模型估计中广泛使用。[{cite}`li2010indirect`]
间接推断法的理论研究表明了它的良好性质。如果选择了合适的binding函数，那么模型的偏差将会显著减少(bias deduction)。[{cite}`phillips2012folklore`]


间接推断的基本思路是从辅助模型(*auxiliary model*)中找出结构参数。

**辅助模型** (*auxiliary model*): 通常是一个简化型的回归模型。

简化型的回归模型忽略了背后的经济学结构，是单纯的统计学过程。因此，简化的回归模型处理起来便捷不少。binding函数( *binding function* )是简化型参数空间与结构型参数空间的一对一映射。估计出简化型的参数之后，我们就可以根据binding函数找出结构型参数。如果简化型参数可以用封闭形式表示，我们就可以利用解析形式匹配理论预测和实际的结果。[{cite}`shi2018structural`] 不过在大多数的情况下，结构型模型得出的简化型并没有封闭形式表达式，所以我们必须用模拟法计算。                                                         

辅助模型的选择并不唯一。在Roy模型的例子中， $\theta$ 是结构参数，构建辅助模型的切入点是 $y_i$ 和 $x_i$ 的线性回归模型。我们可以选 $\hat{b}=(\hat{b}_1,\hat{b}_2,\hat{b}_3)'$ 作为简化型的参数，其中

$$
\begin{align}
\hat{b}_1 & =  (X'X)^{-1}X'y \\
\hat{b}_2 & =  n^{-1}\sum_{y_i=1} (y_i - x_i' b_1)^2 = n^{-1}\sum_{y_i=1} (1 - x_i' b_1)^2  \\
\hat{b}_3 & =  n^{-1}\sum_{y_i=0} (y_i - x_i' b_1)^2 = n^{-1}\sum_{y_i=0} (x_i' b_1)^2.
\end{align}
$$

在此， $\hat{b}_1$ 与 $\beta$ 有关，而 $(\hat{b}_2,\hat{b}_3)$
取决于 $(\sigma_1,\sigma_{12})$ 。

现在我们讨论结构型的参数。给定测试值 $\theta$ ，模型是参数化的，我们就可以人为模拟 $x_i$ 的条件误差 $(e_{i1}^*, e_{i2}^*)$ 。在每次的模拟中，我们可以决定 $y_i^*$ ，进而在给定人造数据后估计简化型参数 $\hat{b}^*=\hat{b}^*(\theta)$ 。$b(\theta)$ 是binding函数。进行 $S$ 次模拟后，我们可以测量 $\hat{b}$ 和 $\hat{b}^*$ 的距离

$$
Q(\theta) = \left(\hat{b} - S^{-1} \sum_{s=1}^S \hat{b}^*(\theta)^{s} \right)'
W \left(\hat{b} - S^{-1} \sum_{s=1}^S \hat{b}^*(\theta)^{s}\right)
$$

其中 $s$ 表示模拟，而 $W$ 是正定权重矩阵。
间接推断估计量是 $\hat{\theta} = \arg\min_{\theta} Q(\theta).$
也就是说，我们寻找真实数据与人造数据的简化型参数差异最小的 $\theta$ 。


## 马尔可夫链-蒙特卡罗法 (Markov Chain Monte Carlo)

如果已知累积分布函数 $F(X)$ ，我们就能生成服从分布的随机变量，从而可以很容易算出 $X = F^{-1}(U)$ ，其中 $U$ 是从 $\mathrm{Uniform}(0,1)$ 中随机抽取的。该 $X$ 服从 $F(X)$ 的分布。

如果概率密度函数 $f(X)$ 已知，我们就能用 **重要性采样法** (*importance sampling*)生成所需的样本。
 [Metropolis-Hastings 算法](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm)
 (MH算法) 就是这样的一种方法。
MH 是其中一种 [Markov Chain Monte Carlo](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo)方法。 
它可以通过 R 拓展包 `mcmc`实现。
[这一网站](https://chi-feng.github.io/mcmc-demo/) 展示MCMC的使用样例。

### Metropolis-Hastings 算法

MH背后的原理需要长导数，但是算法的实现十分简单直接。在这里，我们用MH算法生成正态分布观测值的样本($\mu = 1$ ， $\sigma = 0.5$)。

在函数 `metrop` 中我们提供了

$$
\log f(x) = -\frac{1}{2} \log (2\pi) - \log \sigma - \frac{1}{2\sigma^2} (x-\mu)^2,
$$

密度的对数。由于第一项与参数无关，我们可以将之删去。


In [ ]:
h <- function(x, mu = 1, sd = 0.5) {
  y <- -log(sd) - (x - mu)^2 / (2 * sd^2)
} # un-normalized function (doesn't need to integrate as 1)

out <- mcmc::metrop(obj = h, initial = 0, nbatch = 100, nspac = 1)

par(mfrow = c(3, 1))
par(mar = c(2, 2, 1, 1))
plot(out$batch, type = "l") # a time series with flat steps

out <- mcmc::metrop(obj = h, initial = 0, nbatch = 100, nspac = 10)
plot(out$batch, type = "l") # a time series looks like a white noise

out <- mcmc::metrop(obj = h, initial = 0, nbatch = 10000, nspac = 10)
plot(density(out$batch), main = "", lwd = 2)

xbase <- seq(-2, 2, 0.01)
ynorm <- dnorm(xbase, mean = 1, sd = 0.5)
lines(x = xbase, y = ynorm, type = "l", col = "red")


程序生成的图表包含了三个面板。第一个是一个时间序列，其中所有观测值的边际分布都服从 $N(1,0.5^2)$ 。时间依赖性是可视的；我们也能够观测到当马尔可夫链拒绝新**建议**，两个时期的值没有更新的平坦区(flat regions)。 为了减少时间依赖性，第二个面板在马尔可夫链上每隔十个观测值就收集一次时间序列。在这次的子表中，序列相关性就降低了。第三个面板比较了模拟的观测值的kernel密度(黑线)和 $N(1,0.5^2)$ 的概率密度函数(红线)。


### 拉普拉斯类估计量: MCMC的实际应用

对于某些计量估计量来说，找到全局最优解是很困难的，因为目标函数的行为(behavior)是不规则的。 {cite}`chernozhukov2003mcmc's` 提出了拉普拉斯类估计量(*Laplace-type estimator* LTE) 和夸西-贝叶斯估计量(Quasi-Bayesian estimator QBE)， 它可以规避最优化计算中的棘手问题。 LTE把目标函数的极值估计量转化为概率权重:

$$
f_n (\theta) = \frac{\exp(-L_n(\theta))\pi(\theta)}{\int_{\Theta} \exp(-L_n(\theta))\pi(\theta)}
$$

其中 $L_n(\theta)$ 是目标函数 (比如说OLS、(负) 对数似然或者广义矩估计)， $\pi(\theta)$ 是先验分布(prior distribution)的密度。
目标函数值越小，它的权重就越大。指数转换方式来自于 [Laplace approximation](https://en.wikipedia.org/wiki/Laplace's_method).

我们用MCMC来模拟 $\theta$ 的分布。从贝叶斯学派的视角来看， $f_n(\theta)$ 是一个后验分布(posterior distribution)。
不过，{cite}`chernozhukov2003mcmc`用该分布进行了经典的估计与推断。他们通过频率学派的理论为计算过程提供了理论支撑。 一旦 $f_n(\theta)$ 已知，
那么点估计量 **渐近地** 等同于它在二次损失函数下的均值，也等同于它在绝对值损失函数下的中位数。

下面的程序在线性回归模型中比较了OLS估计量和LTE估计量。


In [ ]:
library(magrittr)
# DGP
n <- 500
b0 <- c(.1, .1)
X <- cbind(1, rnorm(n))
Y <- X %*% b0 + rnorm(n)

b2_OLS <- (lm(Y ~ -1 + X) %>% summary() %>% coef())[2, ]
# "-1" in lm( ) because X has contained intercept
print(cat(
  "The OLS point est =", b2_OLS[1], " sd = ", b2_OLS[2],
  " \n and C.I.=(", c(b2_OLS[1] - 1.96 * b2_OLS[2], b2_OLS[1] + 1.96 * b2_OLS[2]), ")"
))

# Laplace-type estimator
L <- function(b) -0.5 * sum((Y - X %*% b)^2) - 0.5 * crossprod(b - c(0, 0))
# notice the "minus" sign of the OLS objective function
# here we use a normal prior around (0,0).
# results are very similar if we replace it with a flat prior so that
# L <- function(b) -0.5*sum((Y - X %*% b)^2)

nbatch <- 10000
out <- mcmc::metrop(obj = L, initial = c(0, 0), nbatch = nbatch, nspac = 20)

# summarize the estimation
bhat2 <- out$batch[-(1:round(nbatch / 10)), 2] # remove the burn in
bhat2_point <- mean(bhat2)
bhat2_sd <- sd(bhat2)
bhat2_CI <- quantile(bhat2, c(.025, .975))

# compare with OLS
print(cat(
  "The posterior mean =", bhat2_point, " sd = ", bhat2_sd,
  " \n and C.I.=(", bhat2_CI, ")"
))

plot(density(bhat2), main = "posterior from normal prior")


## 后续写作计划

* EM algorithm



## 拓展阅读

* {cite}`li2010indirect`
* Peng (2018), [Advanced Statistical Computing](https://bookdown.org/rdpeng/advstatcomp/) 第五章和第七章1-2节。



## 参考文献
